In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
import os
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
plt.style.use('seaborn-v0_8-whitegrid')  # 使用seaborn样式

# 创建中文字体对象
font_path = 'C:/Windows/Fonts/simhei.ttf'
if os.path.exists(font_path):
    chinese_font = fm.FontProperties(fname=font_path)
    print("使用SimHei字体文件路径")
else:
    # 回退到rcParams方法
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    chinese_font = None
    print("使用rcParams设置SimHei字体")

# 创建输出目录
output_dir = 'dengue_forecast_results'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 读取处理过的数据
print("=== 开始加载数据 ===")
df = pd.read_csv('../pre_data/processed_dengue_data.csv')

# 确保日期列为datetime类型
df['Date'] = pd.to_datetime(df['Date'])

print(f"原始数据形状: {df.shape}")
print(f"数据时间范围: {df['Date'].min()} 至 {df['Date'].max()}")
print("数据加载完成！\n")

使用SimHei字体文件路径
=== 开始加载数据 ===
原始数据形状: (1020, 8)
数据时间范围: 2016-01-01 00:00:00 至 2020-12-01 00:00:00
数据加载完成！



In [3]:
# 1. 准备全国和主要地区的时间序列数据
print("1. 准备时间序列数据")

# 1.1 全国总数据
national_ts = df.groupby('Date')['Dengue_Cases'].sum().reset_index()
national_ts.set_index('Date', inplace=True)

# 1.2 找出前3个高发地区
top_regions = df.groupby('Region')['Dengue_Cases'].sum().sort_values(ascending=False).head(3).index.tolist()
print(f"  前3个高发地区: {', '.join(top_regions)}")

# 1.3 创建主要地区的时间序列数据
region_ts = {}
for region in top_regions:
    region_data = df[df['Region'] == region].groupby('Date')['Dengue_Cases'].sum()
    region_ts[region] = region_data

1. 准备时间序列数据
  前3个高发地区: Region IV-A, Region III, Region VI


In [4]:
# 2. 对全国数据进行时间序列分解
print("\n2. 对全国数据进行时间序列分解")

# 确保索引是按月排序的等间隔时间序列
national_monthly = national_ts.asfreq('MS')
# 填充可能的缺失值
national_monthly = national_monthly.fillna(national_monthly.bfill())

# 进行时间序列分解
decomposition = seasonal_decompose(national_monthly, model='multiplicative', period=12)

# 创建分解图
fig, axes = plt.subplots(4, 1, figsize=(14, 16), sharex=True)

# 原始数据
decomposition.observed.plot(ax=axes[0], color='blue')
if chinese_font is not None:
    axes[0].set_title('观察值', fontproperties=chinese_font, fontsize=14)
    axes[0].set_ylabel('病例数', fontproperties=chinese_font)
else:
    axes[0].set_title('观察值', fontsize=14)
    axes[0].set_ylabel('病例数')

# 趋势
decomposition.trend.plot(ax=axes[1], color='green')
if chinese_font is not None:
    axes[1].set_title('趋势', fontproperties=chinese_font, fontsize=14)
    axes[1].set_ylabel('趋势', fontproperties=chinese_font)
else:
    axes[1].set_title('趋势', fontsize=14)
    axes[1].set_ylabel('趋势')

# 季节性
decomposition.seasonal.plot(ax=axes[2], color='red')
if chinese_font is not None:
    axes[2].set_title('季节性', fontproperties=chinese_font, fontsize=14)
    axes[2].set_ylabel('季节因子', fontproperties=chinese_font)
else:
    axes[2].set_title('季节性', fontsize=14)
    axes[2].set_ylabel('季节因子')

# 残差
decomposition.resid.plot(ax=axes[3], color='purple')
if chinese_font is not None:
    axes[3].set_title('残差', fontproperties=chinese_font, fontsize=14)
    axes[3].set_ylabel('残差', fontproperties=chinese_font)
    axes[3].set_xlabel('日期', fontproperties=chinese_font)
else:
    axes[3].set_title('残差', fontsize=14)
    axes[3].set_ylabel('残差')
    axes[3].set_xlabel('日期')

fig.tight_layout()
plt.savefig(f'{output_dir}/national_time_series_decomposition.png', dpi=300)
print(f"  保存图表：{output_dir}/national_time_series_decomposition.png")
plt.close()


2. 对全国数据进行时间序列分解
  保存图表：dengue_forecast_results/national_time_series_decomposition.png


In [5]:
# 3. 对主要地区数据进行时间序列分解
print("\n3. 对主要地区数据进行时间序列分解")

for region in top_regions:
    # 获取区域数据并转换为月度频率
    region_monthly = region_ts[region].asfreq('MS')
    # 填充可能的缺失值
    region_monthly = region_monthly.fillna(region_monthly.bfill())
    
    # 进行时间序列分解
    region_decomposition = seasonal_decompose(region_monthly, model='multiplicative', period=12)
    
    # 创建分解图
    fig, axes = plt.subplots(4, 1, figsize=(14, 16), sharex=True)
    
    # 原始数据
    region_decomposition.observed.plot(ax=axes[0], color='blue')
    if chinese_font is not None:
        axes[0].set_title(f'{region} - 观察值', fontproperties=chinese_font, fontsize=14)
        axes[0].set_ylabel('病例数', fontproperties=chinese_font)
    else:
        axes[0].set_title(f'{region} - 观察值', fontsize=14)
        axes[0].set_ylabel('病例数')
    
    # 趋势
    region_decomposition.trend.plot(ax=axes[1], color='green')
    if chinese_font is not None:
        axes[1].set_title('趋势', fontproperties=chinese_font, fontsize=14)
        axes[1].set_ylabel('趋势', fontproperties=chinese_font)
    else:
        axes[1].set_title('趋势', fontsize=14)
        axes[1].set_ylabel('趋势')
    
    # 季节性
    region_decomposition.seasonal.plot(ax=axes[2], color='red')
    if chinese_font is not None:
        axes[2].set_title('季节性', fontproperties=chinese_font, fontsize=14)
        axes[2].set_ylabel('季节因子', fontproperties=chinese_font)
    else:
        axes[2].set_title('季节性', fontsize=14)
        axes[2].set_ylabel('季节因子')
    
    # 残差
    region_decomposition.resid.plot(ax=axes[3], color='purple')
    if chinese_font is not None:
        axes[3].set_title('残差', fontproperties=chinese_font, fontsize=14)
        axes[3].set_ylabel('残差', fontproperties=chinese_font)
        axes[3].set_xlabel('日期', fontproperties=chinese_font)
    else:
        axes[3].set_title('残差', fontsize=14)
        axes[3].set_ylabel('残差')
        axes[3].set_xlabel('日期')
    
    fig.tight_layout()
    plt.savefig(f'{output_dir}/{region}_time_series_decomposition.png', dpi=300)
    print(f"  保存图表：{output_dir}/{region}_time_series_decomposition.png")
    plt.close()


3. 对主要地区数据进行时间序列分解
  保存图表：dengue_forecast_results/Region IV-A_time_series_decomposition.png
  保存图表：dengue_forecast_results/Region III_time_series_decomposition.png
  保存图表：dengue_forecast_results/Region VI_time_series_decomposition.png


In [6]:
# 4. 实现季节性平均预测模型（基准模型）
print("\n4. 实现季节性平均预测模型（基准模型）")

def seasonal_average_forecast(ts, period=12, test_size=12, predict_periods=12):
    """
    使用季节性平均方法进行预测
    
    参数:
    ts: 时间序列数据
    period: 季节性周期
    test_size: 测试集大小
    predict_periods: 预测未来的周期数
    
    返回:
    train: 训练集
    test: 测试集
    forecast: 预测结果
    future_forecast: 未来预测
    """
    # 分割训练集和测试集
    train = ts[:-test_size]
    test = ts[-test_size:]
    
    # 计算每个季节的平均值
    seasonal_means = {}
    
    for i in range(period):
        # 获取所有第i个季节的值 (0-indexed)
        season_values = train.iloc[i::period]
        # 计算平均值
        seasonal_means[i] = season_values.mean()
    
    # 创建预测（对于测试集）
    forecast = pd.Series(index=test.index)
    
    for i, idx in enumerate(test.index):
        # 确定当前季节
        season = i % period
        # 使用对应季节的平均值作为预测
        forecast[idx] = seasonal_means[season]
    
    # 创建未来预测
    last_date = test.index[-1]
    future_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), 
                                periods=predict_periods, freq='MS')
    
    future_forecast = pd.Series(index=future_dates)
    
    for i, idx in enumerate(future_dates):
        # 确定当前季节
        season = (test_size + i) % period
        # 使用对应季节的平均值作为预测
        future_forecast[idx] = seasonal_means[season]
    
    return train, test, forecast, future_forecast


4. 实现季节性平均预测模型（基准模型）


In [7]:
# 5. 设置评估指标
print("\n5. 设置评估指标")

def evaluate_forecast(actual, predicted):
    """
    计算各种评估指标
    
    参数:
    actual: 实际值
    predicted: 预测值
    
    返回:
    metrics: 包含各种评估指标的字典
    """
    # 均方根误差 (RMSE)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    
    # 平均绝对误差 (MAE)
    mae = mean_absolute_error(actual, predicted)
    
    # 平均绝对百分比误差 (MAPE)
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    
    # 决定系数 (R²)
    r2 = r2_score(actual, predicted)
    
    # 汇总结果
    metrics = {
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'R²': r2
    }
    
    return metrics



5. 设置评估指标


In [8]:
# 6. 对全国和主要地区数据进行季节性平均预测
print("\n6. 对全国和主要地区数据进行季节性平均预测")

# 6.1 全国数据预测
national_train, national_test, national_forecast, national_future = seasonal_average_forecast(
    national_monthly['Dengue_Cases'], period=12, test_size=12, predict_periods=12)

# 评估全国预测结果
national_metrics = evaluate_forecast(national_test, national_forecast)

print("  全国预测评估指标:")
for metric, value in national_metrics.items():
    print(f"    {metric}: {value:.4f}")

# 保存评估结果
metrics_df = pd.DataFrame([national_metrics], index=['全国'])

# 6.2 主要地区数据预测
for region in top_regions:
    region_monthly = region_ts[region].asfreq('MS')
    region_monthly = region_monthly.fillna(region_monthly.bfill())
    
    region_train, region_test, region_forecast, region_future = seasonal_average_forecast(
        region_monthly, period=12, test_size=12, predict_periods=12)
    
    # 评估地区预测结果
    region_metrics = evaluate_forecast(region_test, region_forecast)
    
    print(f"\n  {region}预测评估指标:")
    for metric, value in region_metrics.items():
        print(f"    {metric}: {value:.4f}")
    
    # 添加到评估结果表
    metrics_df = pd.concat([metrics_df, pd.DataFrame([region_metrics], index=[region])])
    
    # 可视化预测结果
    plt.figure(figsize=(14, 7))
    
    plt.plot(region_train.index, region_train.values, 'b-', label='训练数据')
    plt.plot(region_test.index, region_test.values, 'g-', label='实际测试数据')
    plt.plot(region_forecast.index, region_forecast.values, 'r--', label='预测值')
    plt.plot(region_future.index, region_future.values, 'm--', label='未来预测')
    
    if chinese_font is not None:
        plt.title(f'{region} - 季节性平均预测', fontproperties=chinese_font, fontsize=16)
        plt.xlabel('日期', fontproperties=chinese_font, fontsize=12)
        plt.ylabel('登革热病例数', fontproperties=chinese_font, fontsize=12)
        plt.legend(prop=chinese_font)
    else:
        plt.title(f'{region} - 季节性平均预测', fontsize=16)
        plt.xlabel('日期', fontsize=12)
        plt.ylabel('登革热病例数', fontsize=12)
        plt.legend()
    
    plt.grid(True)
    plt.tight_layout()
    
    # 保存图表
    plt.savefig(f'{output_dir}/{region}_seasonal_avg_forecast.png', dpi=300)
    print(f"  保存图表：{output_dir}/{region}_seasonal_avg_forecast.png")
    plt.close()

# 保存评估指标到CSV
metrics_df.to_csv(f'{output_dir}/seasonal_avg_metrics.csv')
print(f"  保存评估指标：{output_dir}/seasonal_avg_metrics.csv")

# 可视化全国预测结果
plt.figure(figsize=(14, 7))

plt.plot(national_train.index, national_train.values, 'b-', label='训练数据')
plt.plot(national_test.index, national_test.values, 'g-', label='实际测试数据')
plt.plot(national_forecast.index, national_forecast.values, 'r--', label='预测值')
plt.plot(national_future.index, national_future.values, 'm--', label='未来预测')

if chinese_font is not None:
    plt.title('全国 - 季节性平均预测', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('日期', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('登革热病例数', fontproperties=chinese_font, fontsize=12)
    plt.legend(prop=chinese_font)
else:
    plt.title('全国 - 季节性平均预测', fontsize=16)
    plt.xlabel('日期', fontsize=12)
    plt.ylabel('登革热病例数', fontsize=12)
    plt.legend()

plt.grid(True)
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/national_seasonal_avg_forecast.png', dpi=300)
print(f"  保存图表：{output_dir}/national_seasonal_avg_forecast.png")
plt.close()


6. 对全国和主要地区数据进行季节性平均预测
  全国预测评估指标:
    RMSE: 20474.7011
    MAE: 15838.7500
    MAPE: 326.4143
    R²: -11.7319

  Region IV-A预测评估指标:
    RMSE: 3530.0161
    MAE: 2599.0000
    MAPE: 993.0969
    R²: -11.4307
  保存图表：dengue_forecast_results/Region IV-A_seasonal_avg_forecast.png

  Region III预测评估指标:
    RMSE: 1998.1159
    MAE: 1472.7708
    MAPE: 164.8512
    R²: -3.5607
  保存图表：dengue_forecast_results/Region III_seasonal_avg_forecast.png

  Region VI预测评估指标:
    RMSE: 2971.8101
    MAE: 2018.0833
    MAPE: 820.8395
    R²: -169.9661
  保存图表：dengue_forecast_results/Region VI_seasonal_avg_forecast.png
  保存评估指标：dengue_forecast_results/seasonal_avg_metrics.csv
  保存图表：dengue_forecast_results/national_seasonal_avg_forecast.png


In [9]:
# 1. 准备Prophet模型的数据格式
print("1. 准备Prophet模型的数据格式")

def prepare_prophet_data(ts, column_name='y'):
    """
    将时间序列数据转换为Prophet需要的格式
    
    参数:
    ts: 时间序列数据，索引为日期
    column_name: Prophet模型中的目标列名
    
    返回:
    prophet_df: Prophet格式的数据框
    """
    prophet_df = pd.DataFrame()
    prophet_df['ds'] = ts.index
    prophet_df[column_name] = ts.values
    
    return prophet_df

1. 准备Prophet模型的数据格式


In [10]:
# 2. 对全国数据使用Prophet模型
print("\n2. 对全国数据使用Prophet模型")

# 准备全国数据
national_prophet_df = prepare_prophet_data(national_monthly['Dengue_Cases'])

# 分割训练集和测试集
train_df = national_prophet_df.iloc[:-12].copy()
test_df = national_prophet_df.iloc[-12:].copy()

# 创建Prophet模型
model = Prophet(
    yearly_seasonality=True,  # 年度季节性
    weekly_seasonality=False,  # 无需周度季节性
    daily_seasonality=False,   # 无需日度季节性
    seasonality_mode='multiplicative'  # 使用乘法季节性
)

# 拟合模型
model.fit(train_df)

# 创建预测期
future = model.make_future_dataframe(periods=24, freq='MS')  # 12个月测试集+12个月未来预测

# 进行预测
forecast = model.predict(future)

# 提取测试集的预测结果
test_forecast = forecast.iloc[-24:-12]

# 评估模型
prophet_metrics = evaluate_forecast(test_df['y'].values, test_forecast['yhat'].values)

print("  全国Prophet模型评估指标:")
for metric, value in prophet_metrics.items():
    print(f"    {metric}: {value:.4f}")

# 创建Prophet模型评估指标DataFrame
prophet_metrics_df = pd.DataFrame([prophet_metrics], index=['全国'])

# 可视化全国Prophet预测结果
fig, ax = plt.subplots(figsize=(14, 7))

# 训练数据
ax.plot(train_df['ds'], train_df['y'], 'b-', label='训练数据')
# 测试数据
ax.plot(test_df['ds'], test_df['y'], 'g-', label='实际测试数据')
# 预测值
ax.plot(forecast['ds'], forecast['yhat'], 'r--', label='Prophet预测')
# 预测区间
ax.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], 
                color='pink', alpha=0.2, label='预测区间')

if chinese_font is not None:
    ax.set_title('全国 - Prophet模型预测', fontproperties=chinese_font, fontsize=16)
    ax.set_xlabel('日期', fontproperties=chinese_font, fontsize=12)
    ax.set_ylabel('登革热病例数', fontproperties=chinese_font, fontsize=12)
    ax.legend(prop=chinese_font)
else:
    ax.set_title('全国 - Prophet模型预测', fontsize=16)
    ax.set_xlabel('日期', fontsize=12)
    ax.set_ylabel('登革热病例数', fontsize=12)
    ax.legend()

plt.grid(True)
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/national_prophet_forecast.png', dpi=300)
print(f"  保存图表：{output_dir}/national_prophet_forecast.png")
plt.close()

00:52:06 - cmdstanpy - INFO - Chain [1] start processing



2. 对全国数据使用Prophet模型


00:52:07 - cmdstanpy - INFO - Chain [1] done processing


  全国Prophet模型评估指标:
    RMSE: 58940.3339
    MAE: 49954.3806
    MAPE: 1018.0939
    R²: -104.5074
  保存图表：dengue_forecast_results/national_prophet_forecast.png


In [11]:
# 3. 查看Prophet模型的季节性组件
print("\n3. 查看Prophet模型的季节性组件")

# 创建季节性分解图
fig = model.plot_components(forecast)

# 保存季节性分解图
fig.savefig(f'{output_dir}/national_prophet_components.png', dpi=300)
print(f"  保存图表：{output_dir}/national_prophet_components.png")
plt.close()



3. 查看Prophet模型的季节性组件
  保存图表：dengue_forecast_results/national_prophet_components.png


In [12]:
# 4. 对主要地区使用Prophet模型
print("\n4. 对主要地区使用Prophet模型")

for region in top_regions:
    print(f"\n  处理地区: {region}")
    
    # 准备地区数据
    region_monthly = region_ts[region].asfreq('MS')
    region_monthly = region_monthly.fillna(region_monthly.bfill())
    region_prophet_df = prepare_prophet_data(region_monthly)
    
    # 分割训练集和测试集
    region_train_df = region_prophet_df.iloc[:-12].copy()
    region_test_df = region_prophet_df.iloc[-12:].copy()
    
    # 创建Prophet模型
    region_model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode='multiplicative'
    )
    
    # 拟合模型
    region_model.fit(region_train_df)
    
    # 创建预测期
    region_future = region_model.make_future_dataframe(periods=24, freq='MS')
    
    # 进行预测
    region_forecast = region_model.predict(region_future)
    
    # 提取测试集的预测结果
    region_test_forecast = region_forecast.iloc[-24:-12]
    
    # 评估模型
    region_prophet_metrics = evaluate_forecast(
        region_test_df['y'].values, region_test_forecast['yhat'].values)
    
    print(f"    {region} Prophet模型评估指标:")
    for metric, value in region_prophet_metrics.items():
        print(f"      {metric}: {value:.4f}")
    
    # 添加到评估指标DataFrame
    prophet_metrics_df = pd.concat([
        prophet_metrics_df, 
        pd.DataFrame([region_prophet_metrics], index=[region])
    ])
    
    # 可视化地区Prophet预测结果
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # 训练数据
    ax.plot(region_train_df['ds'], region_train_df['y'], 'b-', label='训练数据')
    # 测试数据
    ax.plot(region_test_df['ds'], region_test_df['y'], 'g-', label='实际测试数据')
    # 预测值
    ax.plot(region_forecast['ds'], region_forecast['yhat'], 'r--', label='Prophet预测')
    # 预测区间
    ax.fill_between(region_forecast['ds'], region_forecast['yhat_lower'], 
                    region_forecast['yhat_upper'], color='pink', alpha=0.2, label='预测区间')
    
    if chinese_font is not None:
        ax.set_title(f'{region} - Prophet模型预测', fontproperties=chinese_font, fontsize=16)
        ax.set_xlabel('日期', fontproperties=chinese_font, fontsize=12)
        ax.set_ylabel('登革热病例数', fontproperties=chinese_font, fontsize=12)
        ax.legend(prop=chinese_font)
    else:
        ax.set_title(f'{region} - Prophet模型预测', fontsize=16)
        ax.set_xlabel('日期', fontsize=12)
        ax.set_ylabel('登革热病例数', fontsize=12)
        ax.legend()
    
    plt.grid(True)
    plt.tight_layout()
    
    # 保存图表
    plt.savefig(f'{output_dir}/{region}_prophet_forecast.png', dpi=300)
    print(f"    保存图表：{output_dir}/{region}_prophet_forecast.png")
    plt.close()
    
    # 创建地区季节性分解图
    fig = region_model.plot_components(region_forecast)
    
    # 保存季节性分解图
    fig.savefig(f'{output_dir}/{region}_prophet_components.png', dpi=300)
    print(f"    保存图表：{output_dir}/{region}_prophet_components.png")
    plt.close()

# 保存Prophet模型评估指标到CSV
prophet_metrics_df.to_csv(f'{output_dir}/prophet_metrics.csv')
print(f"  保存评估指标：{output_dir}/prophet_metrics.csv")

00:52:42 - cmdstanpy - INFO - Chain [1] start processing



4. 对主要地区使用Prophet模型

  处理地区: Region IV-A


00:52:42 - cmdstanpy - INFO - Chain [1] done processing


    Region IV-A Prophet模型评估指标:
      RMSE: 13812.9023
      MAE: 11683.8398
      MAPE: 4168.0063
      R²: -189.3325
    保存图表：dengue_forecast_results/Region IV-A_prophet_forecast.png


00:52:43 - cmdstanpy - INFO - Chain [1] start processing


...

    Region III Prophet模型评估指标:
      RMSE: 3516.3460
      MAE: 2572.7130
      MAPE: 298.6598
      R²: -13.1244
    保存图表：dengue_forecast_results/Region III_prophet_forecast.png


00:52:45 - cmdstanpy - INFO - Chain [1] start processing


    保存图表：dengue_forecast_results/Region III_prophet_components.png

  处理地区: Region VI


00:52:45 - cmdstanpy - INFO - Chain [1] done processing


    Region VI Prophet模型评估指标:
      RMSE: 12604.6277
      MAE: 8392.4761
      MAPE: 3383.2743
      R²: -3074.5861
    保存图表：dengue_forecast_results/Region VI_prophet_forecast.png
    保存图表：dengue_forecast_results/Region VI_prophet_components.png
  保存评估指标：dengue_forecast_results/prophet_metrics.csv


In [13]:
# 5. 比较两种模型的性能
print("\n5. 比较两种模型的性能")

# 合并季节性平均和Prophet模型的评估指标
all_metrics = pd.concat([
    metrics_df.add_suffix('_SeasonalAvg'),
    prophet_metrics_df.add_suffix('_Prophet')
], axis=1)

# 保存到CSV
all_metrics.to_csv(f'{output_dir}/model_comparison.csv')
print(f"  保存模型比较结果：{output_dir}/model_comparison.csv")

# 可视化比较结果
metrics_to_plot = ['RMSE', 'MAE', 'MAPE']
regions = ['全国'] + top_regions

for metric in metrics_to_plot:
    plt.figure(figsize=(10, 6))
    
    # 提取季节性平均和Prophet模型的指标
    seasonal_values = all_metrics[f'{metric}_SeasonalAvg'].values
    prophet_values = all_metrics[f'{metric}_Prophet'].values
    
    x = np.arange(len(regions))  # x轴位置
    width = 0.35  # 柱状图宽度
    
    # 绘制柱状图
    rects1 = plt.bar(x - width/2, seasonal_values, width, label='季节性平均模型')
    rects2 = plt.bar(x + width/2, prophet_values, width, label='Prophet模型')
    
    # 设置图表属性
    if chinese_font is not None:
        plt.title(f'不同模型的{metric}比较', fontproperties=chinese_font, fontsize=16)
        plt.xlabel('地区', fontproperties=chinese_font, fontsize=12)
        plt.ylabel(metric, fontproperties=chinese_font, fontsize=12)
        plt.xticks(x, regions, fontproperties=chinese_font, rotation=45)
        plt.legend(prop=chinese_font)
    else:
        plt.title(f'不同模型的{metric}比较', fontsize=16)
        plt.xlabel('地区', fontsize=12)
        plt.ylabel(metric, fontsize=12)
        plt.xticks(x, regions, rotation=45)
        plt.legend()
    
    plt.tight_layout()
    
    # 保存图表
    plt.savefig(f'{output_dir}/model_comparison_{metric}.png', dpi=300)
    print(f"  保存图表：{output_dir}/model_comparison_{metric}.png")
    plt.close()


5. 比较两种模型的性能
  保存模型比较结果：dengue_forecast_results/model_comparison.csv
  保存图表：dengue_forecast_results/model_comparison_RMSE.png
  保存图表：dengue_forecast_results/model_comparison_MAE.png
  保存图表：dengue_forecast_results/model_comparison_MAPE.png


In [14]:
# 6. 使用Prophet模型进行未来12个月的预测
print("\n6. 使用Prophet模型进行未来12个月的预测")

# 使用所有数据重新训练Prophet模型
full_national_prophet_df = prepare_prophet_data(national_monthly['Dengue_Cases'])
full_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,
    seasonality_mode='multiplicative'
)
full_model.fit(full_national_prophet_df)

# 预测未来12个月
future_periods = 12
future = full_model.make_future_dataframe(periods=future_periods, freq='MS')
future_forecast = full_model.predict(future)

# 可视化未来预测
plt.figure(figsize=(14, 7))

# 历史数据
plt.plot(full_national_prophet_df['ds'], full_national_prophet_df['y'], 'b-', label='历史数据')
# 预测值
plt.plot(future_forecast['ds'], future_forecast['yhat'], 'r--', label='预测值')
# 预测区间
plt.fill_between(future_forecast['ds'], future_forecast['yhat_lower'], 
                 future_forecast['yhat_upper'], color='pink', alpha=0.2, label='预测区间')

# 标记预测开始点
last_historical_date = full_national_prophet_df['ds'].max()
plt.axvline(x=last_historical_date, color='green', linestyle='--', label='预测开始')

if chinese_font is not None:
    plt.title('全国登革热病例未来12个月预测', fontproperties=chinese_font, fontsize=16)
    plt.xlabel('日期', fontproperties=chinese_font, fontsize=12)
    plt.ylabel('登革热病例数', fontproperties=chinese_font, fontsize=12)
    plt.legend(prop=chinese_font)
else:
    plt.title('全国登革热病例未来12个月预测', fontsize=16)
    plt.xlabel('日期', fontsize=12)
    plt.ylabel('登革热病例数', fontsize=12)
    plt.legend()

plt.grid(True)
plt.tight_layout()

# 保存图表
plt.savefig(f'{output_dir}/national_future_forecast.png', dpi=300)
print(f"  保存图表：{output_dir}/national_future_forecast.png")
plt.close()

# 导出未来预测结果
future_dates = future_forecast.iloc[-future_periods:][['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
future_dates.columns = ['日期', '预测值', '预测下限', '预测上限']
future_dates.to_csv(f'{output_dir}/national_future_forecast.csv', index=False)
print(f"  保存未来预测结果：{output_dir}/national_future_forecast.csv")

00:53:50 - cmdstanpy - INFO - Chain [1] start processing



6. 使用Prophet模型进行未来12个月的预测


00:53:51 - cmdstanpy - INFO - Chain [1] done processing


  保存图表：dengue_forecast_results/national_future_forecast.png
  保存未来预测结果：dengue_forecast_results/national_future_forecast.csv
